# HomeCredit Model Stability

## Importing Preliminaries

In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

import polars as pl
import re

In [36]:
! pip install polars

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.0 MB/s eta 0:00:00m eta 0:00:010:00:01


## Downloading the Data

In [8]:
!kaggle competitions download -c home-credit-credit-risk-model-stability

100%|█████████████████████████████████████▉| 3.14G/3.14G [01:57<00:00, 29.3MB/s]
100%|██████████████████████████████████████| 3.14G/3.14G [01:57<00:00, 28.7MB/s]


In [10]:
! unzip home-credit-credit-risk-model-stability.zip

Archive:  home-credit-credit-risk-model-stability.zip
  inflating: csv_files/test/test_applprev_1_0.csv  
  inflating: csv_files/test/test_applprev_1_1.csv  
  inflating: csv_files/test/test_applprev_1_2.csv  
  inflating: csv_files/test/test_applprev_2.csv  
  inflating: csv_files/test/test_base.csv  
  inflating: csv_files/test/test_credit_bureau_a_1_0.csv  
  inflating: csv_files/test/test_credit_bureau_a_1_1.csv  
  inflating: csv_files/test/test_credit_bureau_a_1_2.csv  
  inflating: csv_files/test/test_credit_bureau_a_1_3.csv  
  inflating: csv_files/test/test_credit_bureau_a_1_4.csv  
  inflating: csv_files/test/test_credit_bureau_a_2_0.csv  
  inflating: csv_files/test/test_credit_bureau_a_2_1.csv  
  inflating: csv_files/test/test_credit_bureau_a_2_10.csv  
  inflating: csv_files/test/test_credit_bureau_a_2_11.csv  
  inflating: csv_files/test/test_credit_bureau_a_2_2.csv  
  inflating: csv_files/test/test_credit_bureau_a_2_3.csv  
  inflating: csv_files/test/test_credit_burea

In [41]:
# Define the directory path
directory = os.getcwd()+'/parquet_files/train/'

# Function to get file sizes in gigabytes
def get_file_sizes(directory):
    file_info = []
    # Iterate through files in the directory
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        # Check if it's a file (not a directory)
        if os.path.isfile(file_path):
            # Get file size in bytes
            size_in_bytes = os.path.getsize(file_path)
            # Convert size to gigabytes
            size_in_gb = size_in_bytes / (1024**2)
            file_info.append({'File': filename, 'Size (MB)': size_in_gb})
    return file_info

# Get file sizes into a list of dictionaries
file_info = get_file_sizes(directory)

# Sort files by size in gigabytes
sorted_files = sorted(file_info, key=lambda x: x['Size (MB)'], reverse=True)

# Display the sorted files
for file_data in sorted_files:
    print(f"File: {file_data['File']}, Size: {file_data['Size (MB)']:.2f} GB")

File: train_credit_bureau_a_1_1.parquet, Size: 174.80 GB
File: train_credit_bureau_a_1_2.parquet, Size: 117.78 GB
File: train_static_0_0.parquet, Size: 108.58 GB
File: train_applprev_1_0.parquet, Size: 102.00 GB
File: train_static_0_1.parquet, Size: 71.23 GB
File: train_applprev_1_1.parquet, Size: 69.49 GB
File: train_credit_bureau_a_1_3.parquet, Size: 68.45 GB
File: train_credit_bureau_a_1_0.parquet, Size: 61.86 GB
File: train_credit_bureau_a_2_5.parquet, Size: 58.16 GB
File: train_credit_bureau_a_2_4.parquet, Size: 46.95 GB
File: train_credit_bureau_a_2_3.parquet, Size: 45.62 GB
File: train_credit_bureau_a_2_6.parquet, Size: 43.09 GB
File: train_person_1.parquet, Size: 41.86 GB
File: train_credit_bureau_a_2_9.parquet, Size: 30.60 GB
File: train_credit_bureau_a_2_2.parquet, Size: 29.25 GB
File: train_static_cb_0.parquet, Size: 29.23 GB
File: train_tax_registry_c_1.parquet, Size: 28.44 GB
File: train_applprev_2.parquet, Size: 27.17 GB
File: train_credit_bureau_a_2_8.parquet, Size: 22.7

In [42]:
# Get the current working directory
directory = os.getcwd()

print("Current working directory:", directory)

Current working directory: /Users/tristanjoshuaalba/sandbox/homecredit-model-stability


In [44]:
pl.scan_parquet(directory + "/parquet_files/train/train_base.parquet")  

<LazyFrame [5 cols, {"case_id": Int64 … "target": Int64}] at 0x12F5C32E0>

In [47]:
df_base = pl.read_parquet(directory + "/parquet_files/train/train_base.parquet")a

In [48]:
df_base.head()

case_id,date_decision,MONTH,WEEK_NUM,target
i64,str,i64,i64,i64
0,"""2019-01-03""",201901,0,0
1,"""2019-01-03""",201901,0,0
2,"""2019-01-04""",201901,0,0
3,"""2019-01-03""",201901,0,0
4,"""2019-01-04""",201901,0,1


In [121]:
[i for i in file_names if i.endswith(("_1","1_*"))]

['train_other_1',
 'train_credit_bureau_a_1_1',
 'train_static_0_1',
 'train_tax_registry_c_1',
 'train_person_1',
 'train_credit_bureau_b_1',
 'train_tax_registry_b_1',
 'train_debitcard_1',
 'train_applprev_1_1',
 'train_tax_registry_a_1',
 'train_deposit_1',
 'train_credit_bureau_a_2_1']

In [128]:
file_names = os.listdir(directory + "/parquet_files/train/")
file_names = [i.split(".")[0] for i in file_names]


depth_0_files = [ 'train_static_0_0'
                 ,'train_static_0_1'
                 ,'train_static_cb_0']

depth_1_files = ['train_applprev_1_0'
                 , 'train_applprev_1_1'
                 , 'train_tax_registry_a_1'
                 , 'train_tax_registry_b_1'
                 , 'train_tax_registry_c_1'
                 , 'train_credit_bureau_a_1_0'
                 , 'train_credit_bureau_a_1_1'
                 , 'train_credit_bureau_a_1_2'
                 , 'train_credit_bureau_a_1_3'
                 , 'train_credit_bureau_b_1'
                 , 'train_other_1'
                 , 'train_person_1'
                 , 'train_deposit_1'
                 , 'train_debitcard_1']

depth_2_files = ['train_credit_bureau_b_2'
                 ,'train_credit_bureau_a_2_0'
                 ,'train_credit_bureau_a_2_1'
                 ,'train_credit_bureau_a_2_2'
                 ,'train_credit_bureau_a_2_3'
                 ,'train_credit_bureau_a_2_4'
                 ,'train_credit_bureau_a_2_5'
                 ,'train_credit_bureau_a_2_6'
                 ,'train_credit_bureau_a_2_7'
                 ,'train_credit_bureau_a_2_8'
                 ,'train_credit_bureau_a_2_9'
                 ,'train_credit_bureau_a_2_10'
                 , 'train_person_2'
                 , 'train_applprev_2']

base_file = ['train_base']

In [131]:
depth_0_files

['train_static_0_0', 'train_static_0_1', 'train_static_cb_0']

In [135]:
d0 = dict()
for file in depth_0_files:
    d0[file+"_df"] = pl.read_parquet(directory+"/parquet_files/train/"+file+".parquet")

In [137]:
d0["train_static_0_0_df"]

case_id,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,applicationscnt_867L,avgdbddpdlast24m_3658932P,avgdbddpdlast3m_4187120P,avgdbdtollast24m_4525197P,avgdpdtolclosure24_3658938P,avginstallast24m_3658937A,avglnamtstart24m_4525187A,avgmaxdpdlast9m_3716943P,avgoutstandbalancel6m_4187114A,avgpmtlast12m_4525200A,bankacctype_710L,cardtype_51L,clientscnt12m_3712952L,clientscnt3m_3712950L,clientscnt6m_3712949L,clientscnt_100L,clientscnt_1022L,clientscnt_1071L,clientscnt_1130L,clientscnt_136L,clientscnt_157L,clientscnt_257L,clientscnt_304L,clientscnt_360L,clientscnt_493L,clientscnt_533L,clientscnt_887L,…,numinstpaidearlyest_4493214L,numinstpaidlastcontr_4325080L,numinstpaidlate1d_3546852L,numinstregularpaid_973L,numinstregularpaidest_4493210L,numinsttopaygr_769L,numinsttopaygrest_4493213L,numinstunpaidmax_3546851L,numinstunpaidmaxest_4493212L,numnotactivated_1143L,numpmtchanneldd_318L,numrejects9m_859L,opencred_647L,paytype1st_925L,paytype_783L,payvacationpostpone_4187118D,pctinstlsallpaidearl3d_427L,pctinstlsallpaidlat10d_839L,pctinstlsallpaidlate1d_3546856L,pctinstlsallpaidlate4d_3546849L,pctinstlsallpaidlate6d_3546844L,pmtnum_254L,posfpd10lastmonth_333P,posfpd30lastmonth_3976960P,posfstqpd30lastmonth_3976962P,previouscontdistrict_112M,price_1097A,sellerplacecnt_915L,sellerplacescnt_216L,sumoutstandtotal_3546847A,sumoutstandtotalest_4493215A,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A,twobodfilling_608L,typesuite_864L,validfrom_1069D
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,bool,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str
0,null,null,1917.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,null,"""OTHER""","""OTHER""",null,null,null,null,null,null,24.0,0.0,0.0,null,"""a55475b1""",null,0.0,0.0,null,null,0.0,0.0,null,"""BO""",null,null
1,null,null,3134.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,null,"""OTHER""","""OTHER""",null,null,null,null,null,null,18.0,0.0,0.0,null,"""a55475b1""",null,0.0,0.0,null,null,0.0,0.0,null,"""BO""",null,null
2,null,null,4937.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,false,"""OTHER""","""OTHER""",null,null,null,null,null,null,36.0,0.0,0.0,null,"""a55475b1""",null,0.0,0.0,null,null,0.0,0.0,null,"""BO""","""AL""",null
3,null,null,4643.6,0.0,0.0,1.0,0.0,2.0,0.0,1.0,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,1.0,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,0.0,0.0,1.0,false,"""OTHER""","""OTHER""",null,null,null,null,null,null,12.0,0.0,0.0,null,"""a55475b1""",null,1.0,1.0,null,null,0.0,0.0,null,"""BO""","""AL""",null
4,null,null,3390.2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,false,"""OTHER""","""OTHER""",null,null,null,null,null,null,24.0,0.0,0.0,null,"""a55475b1""",null,0.0,0.0,null,null,0.0,0.0,null,"""BO""","""AL""",null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2651088,0.0,117624.79,3045.0,4488.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,-1.0,-2.0,0.0,

In [139]:
d0["train_static_0_0_df"].head()

case_id,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,applicationscnt_867L,avgdbddpdlast24m_3658932P,avgdbddpdlast3m_4187120P,avgdbdtollast24m_4525197P,avgdpdtolclosure24_3658938P,avginstallast24m_3658937A,avglnamtstart24m_4525187A,avgmaxdpdlast9m_3716943P,avgoutstandbalancel6m_4187114A,avgpmtlast12m_4525200A,bankacctype_710L,cardtype_51L,clientscnt12m_3712952L,clientscnt3m_3712950L,clientscnt6m_3712949L,clientscnt_100L,clientscnt_1022L,clientscnt_1071L,clientscnt_1130L,clientscnt_136L,clientscnt_157L,clientscnt_257L,clientscnt_304L,clientscnt_360L,clientscnt_493L,clientscnt_533L,clientscnt_887L,…,numinstpaidearlyest_4493214L,numinstpaidlastcontr_4325080L,numinstpaidlate1d_3546852L,numinstregularpaid_973L,numinstregularpaidest_4493210L,numinsttopaygr_769L,numinsttopaygrest_4493213L,numinstunpaidmax_3546851L,numinstunpaidmaxest_4493212L,numnotactivated_1143L,numpmtchanneldd_318L,numrejects9m_859L,opencred_647L,paytype1st_925L,paytype_783L,payvacationpostpone_4187118D,pctinstlsallpaidearl3d_427L,pctinstlsallpaidlat10d_839L,pctinstlsallpaidlate1d_3546856L,pctinstlsallpaidlate4d_3546849L,pctinstlsallpaidlate6d_3546844L,pmtnum_254L,posfpd10lastmonth_333P,posfpd30lastmonth_3976960P,posfstqpd30lastmonth_3976962P,previouscontdistrict_112M,price_1097A,sellerplacecnt_915L,sellerplacescnt_216L,sumoutstandtotal_3546847A,sumoutstandtotalest_4493215A,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A,twobodfilling_608L,typesuite_864L,validfrom_1069D
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,bool,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str
0,null,null,1917.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,null,"""OTHER""","""OTHER""",null,null,null,null,null,null,24.0,0.0,0.0,null,"""a55475b1""",null,0.0,0.0,null,null,0.0,0.0,null,"""BO""",null,null
1,null,null,3134.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,null,"""OTHER""","""OTHER""",null,null,null,null,null,null,18.0,0.0,0.0,null,"""a55475b1""",null,0.0,0.0,null,null,0.0,0.0,null,"""BO""",null,null
2,null,null,4937.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,false,"""OTHER""","""OTHER""",null,null,null,null,null,null,36.0,0.0,0.0,null,"""a55475b1""",null,0.0,0.0,null,null,0.0,0.0,null,"""BO""","""AL""",null
3,null,null,4643.6,0.0,0.0,1.0,0.0,2.0,0.0,1.0,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,1.0,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,0.0,0.0,1.0,false,"""OTHER""","""OTHER""",null,null,null,null,null,null,12.0,0.0,0.0,null,"""a55475b1""",null,1.0,1.0,null,null,0.0,0.0,null,"""BO""","""AL""",null
4,null,null,3390.2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,false,"""OTHER""","""OTHER""",null,null,null,null,null,null,24.0,0.0,0.0,null,"""a55475b1""",null,0.0,0.0,null,null,0.0,0.0,null,"""BO""","""AL""",null


In [141]:
d0["train_static_0_0_df"].shape

(1003757, 168)

In [142]:
d0["train_static_0_0_df"].dtypes

[Int64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 String,
 String,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 String,
 Float64,
 Float64,
 String,
 String,
 String,
 Float64,
 Float64,
 Float64,
 String,
 Float64,
 String,
 Float64,
 Boolean,
 Boolean,
 String,
 String,
 Float64,
 Float64,
 String,
 Float64,
 Float64,
 Boolean,
 Boolean,
 Boolean,
 String,
 String,
 String,
 String,
 Float64,
 String,
 String,
 String,
 Float64,
 Float64,
 Float64,
 String,
 String,
 Float64,
 String,
 String,
 String,
 String,
 String,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 String,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64

In [143]:
d0["train_static_0_0_df"].describe()

statistic,case_id,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,applicationscnt_867L,avgdbddpdlast24m_3658932P,avgdbddpdlast3m_4187120P,avgdbdtollast24m_4525197P,avgdpdtolclosure24_3658938P,avginstallast24m_3658937A,avglnamtstart24m_4525187A,avgmaxdpdlast9m_3716943P,avgoutstandbalancel6m_4187114A,avgpmtlast12m_4525200A,bankacctype_710L,cardtype_51L,clientscnt12m_3712952L,clientscnt3m_3712950L,clientscnt6m_3712949L,clientscnt_100L,clientscnt_1022L,clientscnt_1071L,clientscnt_1130L,clientscnt_136L,clientscnt_157L,clientscnt_257L,clientscnt_304L,clientscnt_360L,clientscnt_493L,clientscnt_533L,…,numinstpaidearlyest_4493214L,numinstpaidlastcontr_4325080L,numinstpaidlate1d_3546852L,numinstregularpaid_973L,numinstregularpaidest_4493210L,numinsttopaygr_769L,numinsttopaygrest_4493213L,numinstunpaidmax_3546851L,numinstunpaidmaxest_4493212L,numnotactivated_1143L,numpmtchanneldd_318L,numrejects9m_859L,opencred_647L,paytype1st_925L,paytype_783L,payvacationpostpone_4187118D,pctinstlsallpaidearl3d_427L,pctinstlsallpaidlat10d_839L,pctinstlsallpaidlate1d_3546856L,pctinstlsallpaidlate4d_3546849L,pctinstlsallpaidlate6d_3546844L,pmtnum_254L,posfpd10lastmonth_333P,posfpd30lastmonth_3976960P,posfstqpd30lastmonth_3976962P,previouscontdistrict_112M,price_1097A,sellerplacecnt_915L,sellerplacescnt_216L,sumoutstandtotal_3546847A,sumoutstandtotalest_4493215A,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A,twobodfilling_608L,typesuite_864L,validfrom_1069D
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str
"""count""",1.003757e6,707071.0,574903.0,1.003757e6,1.003755e6,1.003757e6,1.003757e6,1.003757e6,1.003757e6,1.003757e6,1.003757e6,577626.0,345754.0,218032.0,674485.0,570036.0,69807.0,488464.0,404763.0,196447.0,"""304690""","""125784""",1.003757e6,1.003757e6,1.003757e6,1.003757e6,1.003757e6,1.003757e6,1.003757e6,397.0,1.003757e6,1.003757e6,1.003757e6,1.003757e6,1.003757e6,1.003757e6,…,295381.0,466841.0,683434.0,680415.0,295381.0,683433.0,295381.0,683433.0,295381.0,1.003757e6,1.003757e6,1.003757e6,782997.0,"""1002696""","""1002696""","""1467""",679714.0,677866.0,679714.0,678977.0,678791.0,975103.0,984565.0,939963.0,911303.0,"""1003757""",869385.0,1.003757e6,1.003757e6,687938.0,295381.0,1.003755e6,1.003755e6,138692.0,"""1003712""","""288520""","""119405"""
"""null_count""",0.0,296686.0,428854.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,426131.0,658003.0,785725.0,329272.0,433721.0,933950.0,515293.0,598994.0,807310.0,"""699067""","""877973""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00336e6,0.0,0.0,0.0,0.0,0.0,0.0,…,708376.0,536916.0,320323.0,323342.0,708376.0,320324.0,708376.0,320324.0,708376.0,0.0,0.0,0.0,220760.0,"""1061""","""1061""","""1002290""",324043.0,325891.0,324043.0,324780.0,324966.0,28654.0,19192.0,63794.0,92454.0,"""0""",134372.0,0.0,0.0,315819.0,708376.0,2.0,2.0,865065.0,"""45""","""715237""","""884352"""
"""mean""",1.2169e6,0.054391,50692.545111,3875.597633,1348.498702,0.000027,0.126184,0.420001,1.238874,0.316396,2.54688,29.615871,29.94086,31.970394,44.529247,5117.126961,43699.38738,0.722608,44570.301553,5935.065595,null,null,0.031995,0.014808,0.02148,0.054143,0.093713,0.036431,0.034257,0.052897,0.088635,0.003271,0.081165,0.003575,0.012012,0.07557,…,14.971081,7.761349,5.193795,26.712168,30.395283,5.926073,5.627281,5.391901,5.111564,0.017951,0.020354,0.321495,0.04301,null,null,null,0.603788,0.079324,0.177496,0.11338,0.096961,17.06412,0.016051,0.007732,0.028932,null,33339.94149,0.153756,1.449694,26465.085776,26060.574366,18184.05553,79276.378771,10576.192565,null,null,null
"""std""",696462.441593,8.743984,65457.938573,2920.237199,2687.341477,0.00931,0.445815,2.683453,10.3

In [145]:
print(d0["train_static_0_0_df"].isnull().sum())

AttributeError: 'DataFrame' object has no attribute 'isnull'

In [146]:
print(d0["train_static_0_0_df"].columns)

['case_id', 'actualdpdtolerance_344P', 'amtinstpaidbefduel24m_4187115A', 'annuity_780A', 'annuitynextmonth_57A', 'applicationcnt_361L', 'applications30d_658L', 'applicationscnt_1086L', 'applicationscnt_464L', 'applicationscnt_629L', 'applicationscnt_867L', 'avgdbddpdlast24m_3658932P', 'avgdbddpdlast3m_4187120P', 'avgdbdtollast24m_4525197P', 'avgdpdtolclosure24_3658938P', 'avginstallast24m_3658937A', 'avglnamtstart24m_4525187A', 'avgmaxdpdlast9m_3716943P', 'avgoutstandbalancel6m_4187114A', 'avgpmtlast12m_4525200A', 'bankacctype_710L', 'cardtype_51L', 'clientscnt12m_3712952L', 'clientscnt3m_3712950L', 'clientscnt6m_3712949L', 'clientscnt_100L', 'clientscnt_1022L', 'clientscnt_1071L', 'clientscnt_1130L', 'clientscnt_136L', 'clientscnt_157L', 'clientscnt_257L', 'clientscnt_304L', 'clientscnt_360L', 'clientscnt_493L', 'clientscnt_533L', 'clientscnt_887L', 'clientscnt_946L', 'cntincpaycont9m_3716944L', 'cntpmts24_3658933L', 'commnoinclast6m_3546845L', 'credamount_770A', 'credtype_322L', 'cur

In [147]:
print(d0["train_static_0_0_df"].dtypes)

[Int64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, String, String, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, String, Float64, Float64, String, String, String, Float64, Float64, Float64, String, Float64, String, Float64, Boolean, Boolean, String, String, Float64, Float64, String, Float64, Float64, Boolean, Boolean, Boolean, String, String, String, String, Float64, String, String, String, Float64, Float64, Float64, String, String, Float64, String, String, String, String, String, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, String, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, F